In [1]:
import os
import shutil

files = [
    '../data_final/modified_residues/phosphorylation_positions.csv'
]

for file in files:
    filename = os.path.basename(file)
    shutil.copy2(file, f'./input_files/{filename}')

In [2]:
import pandas as pd
import glob
import os

folder_path = './input_files/'
csv_files = glob.glob(f'{folder_path}*.csv')
all_sequences = set()
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    all_sequences.update(df['sequence'].tolist())
sequence_dict = {}
for seq in all_sequences:
    sequence_dict[seq] = [None] * len(csv_files)
for csv_index, csv_file in enumerate(csv_files):
    df = pd.read_csv(csv_file, dtype={'positions': str})
    for _, row in df.iterrows():
        positions = [int(pos) for pos in row['positions'].split(',')]
        sequence_dict[row['sequence']][csv_index] = positions
result_data = []
for seq, positions in sequence_dict.items():
    result_data.append({'sequence': seq, 'positions': positions})
result_df = pd.DataFrame(result_data)
result_df.to_csv('input_data.csv', index=False)
print(f"Saved {len(result_df)} sequences to input_data.csv")

Saved 1768608 sequences to input_data.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('input_data.csv')

# First split: 90% train, 10% temp (for val+test)
train_df, temp_df = train_test_split(df, test_size=0.1, random_state=42)

# Second split: Split the 10% into 5% val and 5% test
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Save the splits
train_df.to_csv('data/train.csv', index=False)
val_df.to_csv('data/val.csv', index=False)
test_df.to_csv('data/test.csv', index=False)